# binance spot 

- 现货api

## api

### 深度

- /api/v3/depth
- 深度信息
- https://developers.binance.com/docs/zh-CN/binance-spot-api-docs/rest-api/market-data-endpoints

### get accounts

- 获取帐号 现货余额信息


### k lines

- https://developers.binance.com/docs/zh-CN/binance-spot-api-docs/rest-api/market-data-endpoints#k%E7%BA%BF%E6%95%B0%E6%8D%AE


## References

- https://pypi.org/project/python-dotenv/#description

In [28]:
%%time

import os
import time
from binance.spot import Spot
from loguru import logger
import dtale


def get_klines(symbol="BNBUSDT", interval="5m", limit=10):
    client = Spot()

    # Get server timestamp
    print(f"time: {client.time()}")

    # Get last 10 klines of BNBUSDT at 1h interval
    result = client.klines(symbol, interval, limit=limit)
    return result

# 获取数据
ret = get_klines()


time: {'serverTime': 1741507877086}
CPU times: user 4.79 ms, sys: 3.23 ms, total: 8.01 ms
Wall time: 623 ms


In [ ]:
import pandas as pd
import dtale


#
# https://developers.binance.com/docs/zh-CN/binance-spot-api-docs/rest-api/market-data-endpoints#k%E7%BA%BF%E6%95%B0%E6%8D%AE
#
df = pd.DataFrame(
    ret,
    columns=[
        "开盘时间戳",
        "开盘价",
        "最高价",
        "最低价",
        "收盘价(*)",  # (当前K线未结束的即为最新价)
        "成交量",
        "收盘时间戳",
        "成交额",
        "成交笔数",
        "主动买入成交量",
        "主动买入成交额",
        "请忽略该参数",
    ],
)

# 转换开盘时间, 自动补齐时间长度fmt, 并插入一列到 df
df["开盘时间"] = pd.to_datetime(df["开盘时间戳"], unit="ms")
df["收盘时间"] = pd.to_datetime(df["收盘时间戳"], unit="ms")

# 插入到首列位置
df.insert(0, "开盘时间", df.pop("开盘时间"))

dtale.show(df)

In [ ]:
import os
import time
from binance.spot import Spot
from loguru import logger
import dtale


def get_depth(symbol="BNBUSDT", limit=100):
    client = Spot()

    # Get server timestamp
    print(f"time: {client.time()}")

    # Get last 10 klines of BNBUSDT at 1h interval
    result = client.depth(symbol, limit=limit)
    return result


# 获取数据
ret = get_depth()

time: {'serverTime': 1741509648201}


In [61]:
import pandas as pd

df = pd.DataFrame(ret)

df.head()

,lastUpdateId,bids,asks,更新时间
0,13065271217,"[584.37000000, 28.50100000]","[584.38000000, 13.00400000]",1970-06-01 05:14:31.217
1,13065271217,"[584.36000000, 0.03100000]","[584.39000000, 0.03600000]",1970-06-01 05:14:31.217
2,13065271217,"[584.35000000, 0.75600000]","[584.40000000, 0.01800000]",1970-06-01 05:14:31.217
3,13065271217,"[584.34000000, 3.88600000]","[584.41000000, 0.70200000]",1970-06-01 05:14:31.217
4,13065271217,"[584.33000000, 6.31100000]","[584.42000000, 0.03600000]",1970-06-01 05:14:31.217


In [ ]:
import pandas as pd
import os
import time
from binance.spot import Spot
import dtale
import datetime

client = Spot()


# get server timestamp
def get_server_timestamp():
    ts = client.time()
    return ts.get("serverTime")


ts = get_server_timestamp()
print(f"server ts:{ts}")


def get_exchange_info():
    ret = client.exchange_info()
    return ret


# convert ts to datetime
at = datetime.datetime.fromtimestamp(int(ts) / 1000)
at2 = pd.to_datetime(ts, unit="ms")
print(f"server at1:, {at}")
print(f"server at2:, {at2}")


# exchange info
ret = get_exchange_info()

server ts:1741511090046
server at1:, 2025-03-09 17:04:50.046000
server at2:, 2025-03-09 09:04:50.046000


In [ ]:
ratelimits = ret.get("rateLimits")

dtale.show(ratelimits)

In [ ]:
# open orders
ret = client.open_orders()

In [80]:
symbols = ret.get("symbols")

df = pd.DataFrame(symbols)

df.head(5)

,symbol,status,baseAsset,baseAssetPrecision,quoteAsset,quotePrecision,quoteAssetPrecision,baseCommissionPrecision,quoteCommissionPrecision,orderTypes,...,quoteOrderQtyMarketAllowed,allowTrailingStop,cancelReplaceAllowed,isSpotTradingAllowed,isMarginTradingAllowed,filters,permissions,permissionSets,defaultSelfTradePreventionMode,allowedSelfTradePreventionModes
0,ETHBTC,TRADING,ETH,8,BTC,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS, STOP_L...",...,True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...",[],"[[SPOT, MARGIN, TRD_GRP_004, TRD_GRP_005, TRD_...",EXPIRE_MAKER,"[EXPIRE_TAKER, EXPIRE_MAKER, EXPIRE_BOTH]"
1,LTCBTC,TRADING,LTC,8,BTC,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS, STOP_L...",...,True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...",[],"[[SPOT, MARGIN, TRD_GRP_004, TRD_GRP_005, TRD_...",EXPIRE_MAKER,"[EXPIRE_TAKER, EXPIRE_MAKER, EXPIRE_BOTH]"
2,BNBBTC,TRADING,BNB,8,BTC,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS, STOP_L...",...,True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...",[],"[[SPOT, MARGIN, TRD_GRP_004, TRD_GRP_005, TRD_...",EXPIRE_MAKER,"[EXPIRE_TAKER, EXPIRE_MAKER, EXPIRE_BOTH]"
3,NEOBTC,TRADING,NEO,8,BTC,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS, STOP_L...",...,True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...",[],"[[SPOT, MARGIN, TRD_GRP_006, TRD_GRP_008, TRD_...",EXPIRE_MAKER,"[EXPIRE_TAKER, EXPIRE_MAKER, EXPIRE_BOTH]"
4,QTUMETH,TRADING,QTUM,8,ETH,8,8,8,8,"[LIMIT, LIMIT_MAKER, MARKET, STOP_LOSS, STOP_L...",...,True,True,True,True,False,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...",[],"[[SPOT, TRD_GRP_004, TRD_GRP_008, TRD_GRP_009,...",EXPIRE_MAKER,"[EXPIRE_TAKER, EXPIRE_MAKER, EXPIRE_BOTH]"
